In [2]:
from jax import grad
import jax.numpy as np
from jax.numpy import linalg as la
import numpy as nnp
import math

In [3]:
class TT_Tensor:
    def __init__(self, core, d, n, r):
        self.core = core
        self.d = d
        self.n = n
        self.r = r

In [4]:
def tt_svd(A, eps):
    d = len(A.shape)
    N = np.size(A)
    n = A.shape

    C = A # tmp tensor

    G = [] # tt-cores
    r = [] # tt-ranks
    r.append(1)

    for k in range(1, d):
        C = np.reshape(C, (r[k-1] * n[k-1], int(N / (r[k-1] * n[k-1]))))
  
        # calc low-rank approximation
        u, s, v = la.svd(C)
        sum = 0 
        nsize = np.size(s)
        rres = np.size(s)
        for rk in range(0, nsize):
            for m in range(rk+1, nsize):
                sum = sum + (s[m] ** 2)
            if (sum <= (eps ** 2) * la.norm(A)) and (rres > rk):
                rres = rk + 1 
            sum = 0
        r .append(rres) 

        G.append(np.reshape(u[:, :r[k]], (r[k-1], n[k-1], r[k])))
        s = np.diag(s)
        C = np.dot(s[:r[k], :r[k]], v[:r[k], :])
        N = (N * r[k]) / (n[k-1] * r[k-1])
    
    if len(C.shape) == 2:
        C = np.reshape(C, (C.shape[0], C.shape[1], 1))
        r.append(C.shape[2])
        
    G.append(C)
    
    return TT_Tensor(G, d, n, r)

In [5]:
def tt_to_tensor(tt, shape):
    G = tt.core
    B = G[0]
    for i in range(1, len(G)):
        X = np.reshape(G[i], (G[i].shape[0], G[i].shape[1] * G[i].shape[2]))
        B = np.reshape(B, (int(np.size(B) / G[i].shape[0]), G[i].shape[0]))
        B = np.dot(B, X)
    B = np.reshape(B, shape)
    return B

In [6]:
def tt_sum(tt1, tt2):
    if tt1.d == tt2.d:
        d = tt1.d
    else:
        raise Exception("Different dimensions of tensors.")
    
    r = [r1 + r2 for r1, r2 in zip(tt1.r, tt2.r)]
    
    if tt1.r[0] == tt2.r[0]:
        r[0] = tt1.r[0]
    else:
        raise Exception("Different sizes of first mode.")
        
    if tt1.r[d] == tt2.r[d]:
        r[d] = tt1.r[d]
    else:
        raise Exception("Different sizes of last mode.")
        
    n = tt1.n
    G = []
    
    for i in range(0, d):
        Gi = nnp.zeros((r[i], n[i], r[i+1]))
        Gi[0 : tt1.r[i], :, 0 : tt1.r[i+1]] = tt1.core[i]
        Gi[r[i] - tt2.r[i] : r[i], :, r[i+1] - tt2.r[i+1]: r[i+1]] = tt2.core[i]   
        G.append(Gi)
    
    return TT_Tensor(G, d, n, r)
    

In [134]:
def tt_round(tt, eps):
    delta = eps / math.sqrt(tt.d - 1)
    G = tt.core
    r = tt.r
    rn = []
    for k in range(tt.d-1, 0, -1):
        G[k] = np.reshape(G[k], (tt.r[k], tt.n[k] * tt.r[k+1]))
        G[k] = np.transpose(G[k])
        G[k], R = la.qr(G[k])
        G[k] = np.transpose(G[k])
        G[k-1] = np.einsum('ijk,lk->ijl', G[k-1], R)
        r[k] = G[k-1].shape[2]
        
    for k in range(0, tt.d - 1):
        G[k] = np.reshape(G[k], (tt.r[k] * tt.n[k], tt.r[k+1]))
        G[k+1] = np.reshape(G[k+1], (tt.r[k+1] , tt.n[k+1] * tt.r[k+2]))
        u, s, v = la.svd(G[k])
        rc = s.shape[0]
        p = 0
        for x in s:
            if x < delta:
                x = 0
                if rc > p:
                    rc = p
            p = p + 1
        
        s = np.diag(s)
        u = u[:, :rc]
        s = s[:rc, :rc]
        v = v[:, :rc]
        v = np.dot(v, s)
        G[k] = np.reshape(u, (tt.r[k], tt.n[k], rc))   
        G[k+1] = np.einsum('ij,ik->jk', G[k+1], v)
        r[k+1] = rc
 
    for X in G:
        rn.append(X.shape[0])
    if len(G[tt.d-1]) == 2:
        G[tt.d-1] = np.reshape(G[tt.d-1], (G[tt.d-1].shape[0], G[tt.d-1].shape[0], 1))
        rn.append(1)
    
    return TT_Tensor(G, tt.d, tt.n, rn)

        

In [135]:
# input tensor A
'''
A = np.array([[1/2, 1/3, 1/4], [1/3, 1/4, 1/5], [1/4, 1/5, 1/6]])
A = np.arange(24)
B = A.reshape(4, 3, 2)
A = B
'''

'''
A = np.ones(24)
A = A.reshape(4, 3, 2)
'''

A = []
for i in range(4):
  for j in range(3):
    for q in range(2):
      A.append(1 / (i + j + q + 3))
A = np.asarray(A)
A = A.reshape(4, 3, 2)

print("A:\n", A)

eps = 1e-2 # accuracy

'''

B = tt_to_tensor(tt, A.shape)
print("\nB:\n", B)

print("\nttr:\n")
tts = tt_sum(tt, tt)
for X in tts.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)

print("\nRanks: ", tts.r)

C = tt_to_tensor(tts, A.shape)
print("\nC:\n", C)

D = B + B
print("\nD:\n", D)

print("\nttr:\n")
ttr = tt_round(tt, eps)

for X in ttr.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)

print(tt.r, ttr.r)

B = tt_to_tensor(ttr, A.shape)
print("\nB:\n", B)
print(la.norm(A-B), eps*la.norm(A))
'''

tt = tt_svd(A, eps)

print("\ntt:")
for X in tt.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)

print("\nRanks: ", tt.r)

tts = tt_sum(tt, tt)

print("\ntts:")
for X in tts.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)

print("\nRanks: ", tts.r)

ttr = tt_round(tts, eps)

print("\nttr:")
for X in ttr.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)

print("\nRanks: ", ttr.r)

B = tt_to_tensor(tts, A.shape)
print("\nB = A + A:\n", B)
C = tt_to_tensor(ttr, A.shape)
print("\nC = round(B):\n", C)
print("\n", la.norm(B-C), " <= ", eps*la.norm(B), la.norm(B-C) <= eps*la.norm(B))






A:
 [[[0.33333334 0.25      ]
  [0.25       0.2       ]
  [0.2        0.16666667]]

 [[0.25       0.2       ]
  [0.2        0.16666667]
  [0.16666667 0.14285715]]

 [[0.2        0.16666667]
  [0.16666667 0.14285715]
  [0.14285715 0.125     ]]

 [[0.16666667 0.14285715]
  [0.14285715 0.125     ]
  [0.125      0.11111111]]]

tt:

Core:
 [[[-0.6449674  -0.68758214]
  [-0.5144456   0.09107864]
  [-0.42883325  0.42677027]
  [-0.36805514  0.58034706]]] 
Shape:  (1, 4, 2)

Core:
 [[[-0.6883633   0.40954968]
  [-0.55492496 -0.16872749]
  [-0.46614918 -0.4165662 ]]

 [[-0.02050409  0.66778165]
  [ 0.00807783  0.35927123]
  [ 0.02082663  0.23508413]]] 
Shape:  (2, 3, 2)

Core:
 [[[ 0.704035  ]
  [ 0.57473695]]

 [[-0.01484055]
  [ 0.01817922]]] 
Shape:  (2, 2, 1)

Ranks:  [1, 2, 2, 1]

tts:

Core:
 [[[-0.64496738 -0.68758214 -0.64496738 -0.68758214]
  [-0.5144456   0.09107864 -0.5144456   0.09107864]
  [-0.42883325  0.42677027 -0.42883325  0.42677027]
  [-0.36805514  0.58034706 -0.36805514  0.58